# TP03

Exercice 1

In [ ]:
import numpy as np
import gym
import plotly.graph_objects as go
from traffic_env import TrafficEnvironment

### Create the Environment
env = TrafficEnvironment()
state = env.reset()

for _ in range(10):
    action = 0
    next_state, reward= env.step(action)
    print("Etat: ", next_state, "Reward: ", reward)

    


Etat:  [ 1  2  2 10] Reward:  5
Etat:  [ 0  0  2 10] Reward:  3
Etat:  [ 1  2  4 11] Reward:  0
Etat:  [ 0  2  4 12] Reward:  3
Etat:  [ 0  0  5 13] Reward:  2
Etat:  [ 1  0  5 13] Reward:  0
Etat:  [ 0  1  7 15] Reward:  1
Etat:  [ 0  1  9 16] Reward:  1
Etat:  [ 2  0 10 17] Reward:  1
Etat:  [ 0  1 11 17] Reward:  2


Implémentation de Q-Learning

In [ ]:
Q_table = np.zeros((10, 10, 10, 10, 2))

alpha = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01
episodes = 5000

rewards_per_episode = []

In [24]:
import numpy as np
import matplotlib.pyplot as plt

class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = np.zeros((10, 10, 10, 10, 2, 2))

    def discretize_state(self, state, current_light):
        N = min(state[0], 9)
        S = min(state[1], 9)
        E = min(state[2], 9)
        W = min(state[3], 9)
        return (N, S, E, W, current_light)

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(2) 
        else:
            return np.argmax(self.Q[state])

    def update(self, old_state, action, reward, new_state):
        old_value = self.Q[old_state][action]
        best_next_action = np.argmax(self.Q[new_state])
        td_target = reward + self.gamma * self.Q[new_state][best_next_action]
        self.Q[old_state][action] = old_value + self.alpha * (td_target - old_value)



In [25]:
def train_qlearning(env, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    agent = QLearningAgent(alpha, gamma, epsilon)
    rewards_per_episode = []

    for ep in range(episodes):
        state = env.reset()
        current_light = env.current_light
        discrete_state = agent.discretize_state(state, current_light)
        total_reward = 0

        for t in range(100):
            action = agent.choose_action(discrete_state)
            next_state, reward = env.step(action)
            next_light = env.current_light

            new_discrete_state = agent.discretize_state(next_state, next_light)

            agent.update(discrete_state, action, reward, new_discrete_state)

            discrete_state = new_discrete_state
            total_reward += reward

        rewards_per_episode.append(total_reward)

    return agent, rewards_per_episode


env = TrafficEnvironment()
q_agent, q_learning_rewards = train_qlearning(env, episodes=1000)
print(q_agent.Q)

[[[[[[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[0.00000000e+00 4.09159810e-01]
     [1.58683017e-01 0.00000000e+00]]

    [[3.87187962e-01 0.00000000e+00]
     [2.00000000e-01 0.00000000e+00]]

    ...

    [[4.74746854e-02 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]]


   [[[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[6.69859583e-02 1.14445459e+00]
     [2.55510099e-01 0.00000000e+00]]

    [[1.18016253e+00 0.00000000e+00]
     [6.52207611e-01 0.00000000e+00]]

    ...

    [[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[3.53687438e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[4.26366971e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]]


   [[[0.00000000e+00 0.00000000e+00]
     [5.52305605e-01 0.000000

In [ ]:
import plotly.graph_objects as go
import numpy as np

q_learning_rewards = np.random.randn(500).cumsum()
sarsa_rewards = np.random.randn(500).cumsum()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=q_learning_rewards, mode='lines', name='Q-Learning'))
fig1.update_layout(title="Q-Learning Rewards per Episode",
                   xaxis_title="Episode",
                   yaxis_title="Récompense",
                   template="plotly_dark",
                   width=1000, height=400)
fig1.show()



Implémentation de SARSA

In [27]:
import numpy as np
import matplotlib.pyplot as plt

class SARSAAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = np.zeros((10, 10, 10, 10, 2, 2))

    def discretize_state(self, state, current_light):
        N = min(state[0], 9)
        S = min(state[1], 9)
        E = min(state[2], 9)
        W = min(state[3], 9)
        return (N, S, E, W, current_light)

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(2)
        else:
            return np.argmax(self.Q[state])

    def update(self, old_state, old_action, reward, new_state, new_action):
        old_value = self.Q[old_state][old_action]
        td_target = reward + self.gamma * self.Q[new_state][new_action]
        self.Q[old_state][old_action] = old_value + self.alpha * (td_target - old_value)

In [28]:


def train_sarsa(env, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    agent = SARSAAgent(alpha, gamma, epsilon)
    rewards_per_episode = []

    for ep in range(episodes):
        state = env.reset()
        current_light = env.current_light
        discrete_state = agent.discretize_state(state, current_light)
        action = agent.choose_action(discrete_state)

        total_reward = 0

        for t in range(100):
            next_state, reward = env.step(action)
            next_light = env.current_light
            new_discrete_state = agent.discretize_state(next_state, next_light)

            next_action = agent.choose_action(new_discrete_state)

            agent.update(discrete_state, action, reward, new_discrete_state, next_action)

            discrete_state = new_discrete_state
            action = next_action
            total_reward += reward

        rewards_per_episode.append(total_reward)

    return agent, rewards_per_episode

env = TrafficEnvironment()
sarsa_agent, sarsa_rewards = train_sarsa(env, episodes=1000)
print(sarsa_agent.Q)



[[[[[[4.10076610e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[8.49910642e-02 0.00000000e+00]
     [1.46605205e-01 0.00000000e+00]]

    [[1.20757365e+00 0.00000000e+00]
     [2.57115155e-01 0.00000000e+00]]

    ...

    [[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[0.00000000e+00 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[3.02880428e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]]


   [[[8.65612877e-01 0.00000000e+00]
     [0.00000000e+00 3.43776371e-01]]

    [[9.78947859e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[1.87548021e+00 0.00000000e+00]
     [4.31655588e-01 0.00000000e+00]]

    ...

    [[2.83154426e-02 0.00000000e+00]
     [4.78300000e-01 0.00000000e+00]]

    [[3.83819424e-01 0.00000000e+00]
     [0.00000000e+00 0.00000000e+00]]

    [[1.11572595e-01 1.10930000e+00]
     [0.00000000e+00 0.00000000e+00]]]


   [[[8.37882637e-01 0.00000000e+00]
     [1.00000000e-01 0.000000

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=sarsa_rewards, mode='lines', name='SARSA'))
fig2.update_layout(title="SARSA Rewards per Episode",
                   xaxis_title="Episode",
                   yaxis_title="Récompense",
                   template="plotly_dark",
                   width=1000, height=400)
fig2.show()


Visualisation des Résultats

In [ ]:
cumulative_q = np.cumsum(q_learning_rewards)
cumulative_sarsa = np.cumsum(sarsa_rewards)

fig3 = go.Figure()
fig3.add_trace(go.Scatter(y=cumulative_q, mode='lines', name='Q-Learning (Cumul)'))
fig3.add_trace(go.Scatter(y=cumulative_sarsa, mode='lines', name='SARSA (Cumul)'))
fig3.update_layout(title="Somme des récompenses cumulées",
                   xaxis_title="Épisode",
                   yaxis_title="Récompense cumulée",
                   template="plotly_dark",
                   width=1000, height=400)
fig3.show()



In [ ]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(y=q_learning_rewards, mode='lines', name='Q-Learning'))
fig4.add_trace(go.Scatter(y=sarsa_rewards, mode='lines', name='SARSA'))
fig4.update_layout(title="Comparaison Q-Learning vs SARSA",
                   xaxis_title="Épisode",
                   yaxis_title="Récompense totale par épisode",
                   template="plotly_dark",
                   width=1200, height=500)
fig4.show()


<p align="center">EL HACHYMI Ahmed Yassine</p>